In [4]:
#!/usr/bin/env python3
"""
sne_pipeline.py

pipeline for:
1) Build 3D SN master table from Pantheon+SH0ES.dat
2) Add Galactic coordinates + Galactic Cartesian
3) Run GLS fit with STAT+SYS covariance (Pantheon+SH0ES_STAT+SYS.cov)
4) Save plotting subset (for 3D visualization) + calibrator covariance submatrix

Edit PATHS / COSMO / Z CUTS below if needed.
"""

import numpy as np
import pandas as pd

from astropy.cosmology import FlatLambdaCDM
from astropy.coordinates import SkyCoord
import astropy.units as u

from scipy.linalg import cho_factor, cho_solve


# ==========================
# User settings (edit here)
# ==========================
DATA_PATH = "Pantheon+SH0ES.dat" # available at Pantheon+SH0ES Data Release
COV_PATH  = "Pantheon+SH0ES_STAT+SYS.cov" # available at Pantheon+SH0ES Data Release

OUT_XLSX  = "SNe3D_master_with_galactic.xlsx"
OUT_CSV   = "SNe3D_master_with_galactic.csv"

# Cosmology used ONLY to generate comoving/luminosity/ang. distances for the 3D plot
COSMO_H0  = 73.0
COSMO_OM0 = 0.3

# Fit settings
q0, j0 = -0.55, 1.0
Z_MIN, Z_MAX = 0.0, 0.15
ckms = 2.99792458e5


# ==========================
# Core helpers
# ==========================
def load_pantheon_table(path: str) -> pd.DataFrame:
    df = pd.read_csv(path, sep=r"\s+", comment="#", engine="python")
    return df

def compute_distances_and_cartesian(df: pd.DataFrame, cosmo: FlatLambdaCDM) -> pd.DataFrame:
    z = df["zHD"].to_numpy().astype(float)

    df["D_comoving_Mpc"]   = cosmo.comoving_distance(z).to(u.Mpc).value
    df["D_luminosity_Mpc"] = cosmo.luminosity_distance(z).to(u.Mpc).value
    df["D_angular_Mpc"]    = cosmo.angular_diameter_distance(z).to(u.Mpc).value

    r   = df["D_comoving_Mpc"].to_numpy()
    ra  = np.deg2rad(df["RA"].to_numpy())
    dec = np.deg2rad(df["DEC"].to_numpy())

    # ICRS Cartesian
    df["x_Mpc"] = r * np.cos(dec) * np.cos(ra)
    df["y_Mpc"] = r * np.cos(dec) * np.sin(ra)
    df["z_Mpc"] = r * np.sin(dec)

    return df

def add_galactic_coords_and_cartesian(df: pd.DataFrame) -> pd.DataFrame:
    sky_eq = SkyCoord(ra=df["RA"].values * u.deg, dec=df["DEC"].values * u.deg, frame="icrs")
    gal = sky_eq.galactic

    df["l_deg"] = gal.l.deg
    df["b_deg"] = gal.b.deg

    r = df["D_comoving_Mpc"].to_numpy()
    l = np.deg2rad(df["l_deg"].to_numpy())
    b = np.deg2rad(df["b_deg"].to_numpy())

    df["x_gal_Mpc"] = r * np.cos(b) * np.cos(l)
    df["y_gal_Mpc"] = r * np.cos(b) * np.sin(l)
    df["z_gal_Mpc"] = r * np.sin(b)

    return df

def read_covariance(cov_path: str):
    # Header contains N or "N=1701"
    with open(cov_path, "r") as f:
        header = f.readline().strip()

    try:
        N_total = int(header)
    except ValueError:
        N_total = int(header.split("=")[-1])

    cov_arr = np.loadtxt(cov_path, skiprows=1)
    cov_full = cov_arr.reshape((N_total, N_total)) if cov_arr.ndim == 1 else cov_arr

    # Symmetrize tiny numerical asymmetry
    if not np.allclose(cov_full, cov_full.T, atol=1e-10):
        cov_full = 0.5 * (cov_full + cov_full.T)

    return cov_full, N_total


# ==========================
# GLS fit
# ==========================
def run_gls_fit(master_xlsx_path: str, cov_path: str):
    df_all = pd.read_excel(master_xlsx_path)

    cov_full, N_total = read_covariance(cov_path)

    # Infer offset: master may include leading rows not covered by cov
    offset = len(df_all) - N_total
    if offset < 0:
        raise RuntimeError("Covariance size exceeds master rows — files misaligned?")

    df_all["COV_INDEX"] = np.arange(len(df_all)) - offset
    mask_in_cov = (df_all["COV_INDEX"] >= 0) & (df_all["COV_INDEX"] < N_total)
    df_cov = df_all[mask_in_cov].copy()

    # z cut for the fit
    df_cov = df_cov[(df_cov["zHD"] >= Z_MIN) & (df_cov["zHD"] <= Z_MAX)].copy()

    # Calibrators and HF selection
    is_cal = df_cov["IS_CALIBRATOR"].astype(int).to_numpy() == 1
    is_hf  = (df_cov["USED_IN_SH0ES_HF"].astype(int).to_numpy() == 1) & (~is_cal)

    cal_fit = df_cov[is_cal].copy()
    hf_fit  = df_cov[is_hf].copy()

    # Fit order must match covariance order
    df_fit = pd.concat([cal_fit, hf_fit], ignore_index=False).sort_values("COV_INDEX").copy()

    print(f"[Info] FIT rows: {len(df_fit)} | Cal={int((df_fit['IS_CALIBRATOR']==1).sum())} | HF={int((df_fit['IS_CALIBRATOR']==0).sum())}")

    # Build y and L
    N = len(df_fit)
    y = np.zeros(N)
    L = np.zeros((N, 2))

    cal_mask = (df_fit["IS_CALIBRATOR"].astype(int).to_numpy() == 1)
    hf_mask  = ~cal_mask

    mb = df_fit["m_b_corr"].astype(float).to_numpy()

    # Calibrators: y = m_b_corr - mu_ceph ; L = [1, 0]
    if cal_mask.any():
        # If CEPH_DIST is already distance modulus:
        mu_ceph = df_fit.loc[cal_mask, "CEPH_DIST"].astype(float).to_numpy()
        y[cal_mask] = mb[cal_mask] - mu_ceph
        L[cal_mask, :] = (1.0, 0.0)

    # HF: y = m_b_corr - mu_cz ; L = [1, -1]
    if hf_mask.any():
        z = df_fit.loc[hf_mask, "zHD"].astype(float).to_numpy()
        F = z * (1 + 0.5*(1 - q0)*z - (1/6)*(1 - q0 - 3*q0**2 + j0)*z**2)
        mu_cz = 5*np.log10(ckms * F) + 25.0
        y[hf_mask] = mb[hf_mask] - mu_cz
        L[hf_mask, :] = (1.0, -1.0)

    # Cut covariance to used rows
    idx = df_fit["COV_INDEX"].astype(int).to_numpy()
    C = cov_full[np.ix_(idx, idx)]

    # GLS solve
    choC   = cho_factor(C, overwrite_a=False, check_finite=True)
    Cinvy  = cho_solve(choC, y)
    CinvL  = cho_solve(choC, L)
    Fisher = L.T @ CinvL
    rhs    = L.T @ Cinvy

    choF  = cho_factor(Fisher, overwrite_a=False, check_finite=True)
    q_hat = cho_solve(choF, rhs)
    C_q   = np.linalg.inv(Fisher)

    M_B0, h = q_hat
    sMB0, sh = np.sqrt(np.diag(C_q))
    rho = C_q[0, 1] / (sMB0 * sh)

    H0  = 10**(h/5.0)
    sH0 = (np.log(10)/5.0) * H0 * sh

    print("\n--- GLS fit (STAT+SYS submatrix) ---")
    print(f"M_B^0  = {M_B0:8.4f} ± {sMB0:6.4f} mag")
    print(f"h      = {h:8.4f} ± {sh:6.4f}  (5 log10 H0)")
    print(f"ρ(M_B^0, h) = {rho:6.3f}")
    print(f"H0     = {H0:8.3f} ± {sH0:5.3f} km/s/Mpc")

    res = y - (L @ q_hat)
    print(f"RMS residuals: Cal = {np.std(res[cal_mask]):.3f} mag | HF = {np.std(res[hf_mask]):.3f} mag")

    # Save de-duplicated plotting subset
    cal_raw = df_cov[df_cov["IS_CALIBRATOR"].astype(int) == 1].copy()

    cal_plot = (
        cal_raw[~cal_raw["CID"].astype(str).str.contains("2005df_ANU", case=False, na=False)]
          .sort_values(["CID", "m_b_corr_err_RAW"], ascending=[True, True])
          .drop_duplicates(subset=["CID"], keep="first")
          .copy()
    )

    hf_plot = df_cov[is_hf].copy()
    df_plot = pd.concat([cal_plot, hf_plot], ignore_index=True)
    df_plot.to_parquet("sn_plot_subset_nodup.parquet", index=False)
    print("[Saved] De-duplicated plotting subset -> sn_plot_subset_nodup.parquet")

    # Calibrator-only covariance submatrix
    cal_mask_fit = (df_fit["IS_CALIBRATOR"].astype(int).to_numpy() == 1)
    C_cal = C[np.ix_(cal_mask_fit, cal_mask_fit)]

    idx_cal = df_fit.loc[cal_mask_fit, "COV_INDEX"].to_numpy()
    cid_cal = df_fit.loc[cal_mask_fit, "CID"].astype(str).to_numpy()

    df_cov_cal = pd.DataFrame(C_cal, index=idx_cal, columns=idx_cal)
    sigma_cal = np.sqrt(np.diag(C_cal))
    df_sigma = pd.DataFrame({"COV_INDEX": idx_cal, "CID": cid_cal, "sigma": sigma_cal})

    with pd.ExcelWriter("cov_calibrators_77x77.xlsx") as writer:
        df_cov_cal.to_excel(writer, sheet_name="cov_cal", index=True)
        df_sigma.to_excel(writer, sheet_name="sigma_diag", index=False)

    print("[Saved] Calibrator-only covariance + sigmas -> cov_calibrators_77x77.xlsx")


# ==========================
# Main
# ==========================
def main():
    # Load + minimal filtering (finite coords and z)
    B = load_pantheon_table(DATA_PATH)

    need = [
        "CID", "IDSURVEY", "RA", "DEC", "zHD", "c",
        "m_b_corr", "m_b_corr_err_RAW", "m_b_corr_err_VPEC",
        "CEPH_DIST", "IS_CALIBRATOR", "USED_IN_SH0ES_HF"
    ]
    df = B.loc[np.isfinite(B["RA"]) & np.isfinite(B["DEC"]) & np.isfinite(B["zHD"]), need].copy()

    # Distances + 3D coordinates for visualization
    cosmo = FlatLambdaCDM(H0=COSMO_H0, Om0=COSMO_OM0)
    df = compute_distances_and_cartesian(df, cosmo)
    df = add_galactic_coords_and_cartesian(df)

    # Save master
    df.to_excel(OUT_XLSX, index=False)
    df.to_csv(OUT_CSV, index=False)
    print(f"[Saved] Master -> {OUT_XLSX}")
    print(f"[Saved] Master -> {OUT_CSV}")

    # Run GLS fit
    run_gls_fit(OUT_XLSX, COV_PATH)


if __name__ == "__main__":
    main()


[Saved] Master -> SNe3D_master_with_galactic.xlsx
[Saved] Master -> SNe3D_master_with_galactic.csv
[Info] FIT rows: 354 | Cal=77 | HF=277

--- GLS fit (STAT+SYS submatrix) ---
M_B^0  = -19.2469 ± 0.0299 mag
h      =   9.3324 ± 0.0310  (5 log10 H0)
ρ(M_B^0, h) =  0.957
H0     =   73.534 ± 1.048 km/s/Mpc
RMS residuals: Cal = 0.146 mag | HF = 0.135 mag
[Saved] De-duplicated plotting subset -> sn_plot_subset_nodup.parquet
[Saved] Calibrator-only covariance + sigmas -> cov_calibrators_77x77.xlsx
